In [1]:
import tensorflow as tf
import numpy as np

In [7]:
# the input data is 128+64 random binary vectors of dimension 13
# the output for the last 128 times must be zeros vector of dimension 80 with 1 on 16-th position...
def generate():
    X = np.random.randint(0, 2, (128, 128+64, 13))
    X[:, 128:, :] = 0
    Y = np.zeros((128, 128+64, 80))
    Y[:, :, 16] = 1
    return X, Y

In [11]:
# the input data is 128+64 random binary vectors of dimension 13
# the output data has same dimensions; final 128 of vectors must be same as first 128 of input
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.001
training_steps = 2000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 13       # dimension of one input at moment
timesteps = 128+64   # timesteps in one sequence
num_hidden = 70      # num of lstms in first hidden layer
num_hidden2 = 50     # num of lstms in second hidden layer
num_output = 80      # output dimension

# tf graph input
X = tf.placeholder("float", [batch_size, timesteps, num_input])
Y = tf.placeholder("float", [batch_size, timesteps, num_output])

# loss will be calculated in the loop as sum of losses on each iteration
loss_op = tf.constant(0.0)

# defining first hidden layer
with tf.variable_scope('first_lstm'):
    first_lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    first_lstm_state = tf.zeros([batch_size, num_hidden]), tf.zeros([batch_size, num_hidden])

# defining second hidden layer
with tf.variable_scope("read_lstm"):
    read_lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden2, forget_bias=1.0)
    read_lstm_state = tf.zeros([batch_size, num_hidden2]), tf.zeros([batch_size, num_hidden2])

# parameters of non-linear transformation of second lstm cell outputs to the output
read_w = tf.Variable(tf.random_normal([num_hidden2, num_output]))
read_b = tf.Variable(tf.random_normal([num_output]))

# reshaping data to (timesteps, batch_size, num_input)
_X = tf.unstack(X, timesteps, 1)
_Y = tf.unstack(Y, timesteps, 1)

# time loop
for t, inp, truth in zip(np.arange(timesteps), _X, _Y):
    # moving data through lstm layers
    with tf.variable_scope('first_lstm'):
        first_lstm_output, first_lstm_state = first_lstm(inp, first_lstm_state)
    with tf.variable_scope("read_lstm"):
        read_lstm_output, read_lstm_state = read_lstm(first_lstm_output, read_lstm_state)
    
    # non-linear transformation to vector of history_size length
    read = tf.sigmoid(tf.matmul(read_lstm_output, read_w) + read_b)
    # transformation to "probabilities"-kind weights    
    read_proba = tf.nn.softmax(read, 1)
    
    # calculate loss
    if t >= 64:
        loss_op += tf.reduce_mean(tf.squared_difference(read_proba, truth))

# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                   beta1=0.9,
                                   beta2=0.999,
                                   epsilon=1e-08,
                                   use_locking=False,
                                   name='Adam')
# optimizing
train_op = tf.contrib.layers.optimize_loss(
                loss_op, tf.train.get_global_step(), optimizer=optimizer, learning_rate=learning_rate)

init = tf.global_variables_initializer()

In [12]:
# Start training
config = tf.ConfigProto(log_device_placement=True)  # to output is variable on gpu or cpu
config.gpu_options.per_process_gpu_memory_fraction = 0.4  # BlasGEMM launch failed error fix (seems like not enopugh memory on gpu)
sess = tf.Session(config = config)

# Run the initializer
sess.run(init)

In [13]:
for step in range(0, training_steps):
    # getting data
    batch_x, batch_y = generate()
    # Run optimization step
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
    
    if step % display_step == 0:
        L = sess.run(loss_op, feed_dict={X: batch_x, Y: batch_y})
        print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(L))

print("Optimization Finished!")

Step 0, Minibatch Loss= 1.5822
Step 200, Minibatch Loss= 1.5180
Step 400, Minibatch Loss= 1.5145
Step 600, Minibatch Loss= 1.5143
Step 800, Minibatch Loss= 1.5143
Step 1000, Minibatch Loss= 1.5143
Step 1200, Minibatch Loss= 1.5143
Step 1400, Minibatch Loss= 1.5143
Step 1600, Minibatch Loss= 1.5143
Step 1800, Minibatch Loss= 1.5143
Optimization Finished!
